<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1
# *Data Wrangling and Munging with Pandas*

In this lab we work through reading in files of a range of formats, then manipulating a dataset to handle missing values and performing data profiling.

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [14]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [16]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.26.4
Pandas:  2.2.2


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [18]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'bikeshare.csv'

ANSWER:Requires path - C:\Users\lytton\Downloads\DATA\bikeshare.csv, header should be 0, seperator requires defining.

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [20]:
bikes = pd.read_table(r'\Users\lytton\Downloads\DATA\bikeshare.csv', header = 0, sep = ',')
print(bikes.head())
print()


   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  



Note that we could have used `read_csv()` above. `read_table()` is necessary when `sep` is not the comma character, or we need fine control that `read_csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be dealt with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [22]:
!pip install xlrd>=2.0.1
df = pd.read_excel(r'C:\Users\lytton\Downloads\DATA\Iris.xls', sheet_name = 'Data')
print(df.head())

   Species_No  Petal_width  Petal_length  Sepal_width  Sepal_length  \
0           1          0.2           1.4          3.5           5.1   
1           1          0.2           1.4          3.0           4.9   
2           1          0.2           1.3          3.2           4.7   
3           1          0.2           1.5          3.1           4.6   
4           1          0.2           1.4          3.6           5.0   

  Species_name  
0       Setosa  
1       Setosa  
2       Setosa  
3       Setosa  
4       Setosa  


It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For example, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted column Species_No:

In [24]:
skip_col = [0]
keep_cols = [i for i in range(6) if i not in skip_col]
df = pd.read_excel(r'C:\Users\lytton\Downloads\DATA\Iris.xls', sheet_name = 'Data', usecols = keep_cols)
print(df.head())

   Petal_width  Petal_length  Sepal_width  Sepal_length Species_name
0          0.2           1.4          3.5           5.1       Setosa
1          0.2           1.4          3.0           4.9       Setosa
2          0.2           1.3          3.2           4.7       Setosa
3          0.2           1.5          3.1           4.6       Setosa
4          0.2           1.4          3.6           5.0       Setosa


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [26]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [28]:
url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4           Aberdeen     Scotland  ABZ
 ...              ...          ...  ...
 1573        Zanzibar     Tanzania  ZNZ
 1574        Zaragoza        Spain  ZAZ
 1575       Zhengzhou        China  CGO
 1576        Zhoushan        China  HSN
 1577          Zurich  Switzerland  ZRH
 
 [1578 rows x 3 columns]]

In [29]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,Aberdeen,Scotland,ABZ
...,...,...,...
1573,Zanzibar,Tanzania,ZNZ
1574,Zaragoza,Spain,ZAZ
1575,Zhengzhou,China,CGO
1576,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here are a few examples: https://pandas.pydata.org/docs/user_guide/io.html#io-read-xml

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example:

In [150]:
url = 'https://microsoftedge.github.io/Demos/json-dummy-data/64KB.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,name,language,id,bio,version
0,Adeel Solangi,Sindhi,V59OF92YF627HFY0,Donec lobortis eleifend condimentum. Cras dict...,6.10
1,Afzal Ghaffar,Sindhi,ENTOCR13RSCLZ6KU,"Aliquam sollicitudin ante ligula, eget malesua...",1.88
2,Aamir Solangi,Sindhi,IAKPO3R4761JDRVG,Vestibulum pharetra libero et velit gravida eu...,7.27
3,Abla Dilmurat,Uyghur,5ZVOEPMJUI4MB4EN,Donec lobortis eleifend condimentum. Morbi ac ...,2.53
4,Adil Eli,Uyghur,6VTI8X6LL0MMPJCC,"Vivamus id faucibus velit, id posuere leo. Mor...",6.49


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [155]:
bikes.shape


(17379, 17)

What are the column names in `bikes`?

In [171]:
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [157]:
bikes.dtypes


instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [173]:
bikes.index


RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

### Slicing

Slicing refers to extracting part of a Python object such as a list or dataframe.

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this.
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [179]:
bikes['season']

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [181]:
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [189]:
bikes[['atemp']][0:5]


,atemp
0,0.2879
1,0.2727
2,0.2727
3,0.2879
4,0.2879


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ):

!!! Pandas now highly recommends the use of explicit index calling via either .loc or .iloc. Below are examples of implicit index calling which is being deprecated

In [191]:
bikes[[0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Index([0], dtype='int32')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [197]:
bikes.iloc[[0]]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [203]:
bikes.iloc[:,[1]]


,dteday
0,2011-01-01
1,2011-01-01
2,2011-01-01
3,2011-01-01
4,2011-01-01
...,...
17374,2012-12-31
17375,2012-12-31
17376,2012-12-31
17377,2012-12-31


### Handling Missing Values

What is the Pandas `isnull` function for?

ANSWER:Detect missing values for an array-like object.

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [205]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [209]:
bikes.isnull().sum()


instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [219]:
bikes.columns[bikes.isnull().any()]

Index([], dtype='object')

What is the Numpy object `nan` used for? (Write a descriptive answer.)

ANSWER:Not a number

Write (and verify) a function that performs scalar division with built-in handling of the edge cases (i.e. divide by zero, divide by NaN). Return a value for the edge cases instead of just trapping the error:

In [260]:
import math

def safe_divide(numerator, denominator):
    if denominator == 0:
        return str('inf') 
    
    elif math.isnan(denominator):
        return str('nan') 
    
    # Normal division
    else:
        return numerator / denominator

print(safe_divide(1,0))
print(safe_divide(1,float('nan')))

inf
nan


Apply the Pandas `isna` function to the following data objects:

In [222]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [277]:
print(pd.isna(x))
print(pd.isna(y))

False
True


In [279]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [289]:
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

ANSWER:In pd.read_csv, na_values is used top tell pandas what to substitute NaN values for.

### Data Profiling

#### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [42]:
bikes['season'].value_counts()


season
3    4496
2    4409
1    4242
4    4232
Name: count, dtype: int64

#### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns:

In [69]:
print("instant: " + str(bikes['instant'].min()) + "-" + str(bikes['instant'].max()))
print("dteday: " + str(bikes['dteday'].min()) + "-" + str(bikes['dteday'].max()))
print("windspeed: " + str(bikes['windspeed'].min()) + "-" + str(bikes['windspeed'].max()))

instant: 1-17379
dteday: 2011-01-01-2012-12-31
windspeed: 0.0-0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [81]:
bikes.min()

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                0
workingday             0
weathersit             1
temp                0.02
atemp                0.0
hum                  0.0
windspeed            0.0
casual                 0
registered             0
cnt                    1
dtype: object

In [83]:
bikes.max()

instant            17379
dteday        2012-12-31
season                 4
yr                     1
mnth                  12
hr                    23
holiday                1
weekday                6
workingday             1
weathersit             4
temp                 1.0
atemp                1.0
hum                  1.0
windspeed         0.8507
casual               367
registered           886
cnt                  977
dtype: object

#### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [86]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code:

In [100]:
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))


0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

#### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [103]:
type(bikes['atemp'])

pandas.core.series.Series

In [105]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1515,1516,2011-03-08,1,0,3,12,0,2,1,2,0.36,0.3788,0.23,0.0000,29,70,99
13393,13394,2012-07-17,3,1,7,6,0,2,1,1,0.72,0.6818,0.66,0.1045,16,197,213
3218,3219,2011-05-19,2,0,5,0,0,4,1,1,0.52,0.5000,0.94,0.1045,6,23,29
11543,11544,2012-05-01,2,1,5,4,0,2,1,2,0.52,0.5000,0.72,0.0896,1,7,8
15704,15705,2012-10-21,4,1,10,13,0,0,0,1,0.56,0.5303,0.37,0.2537,243,383,626


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indicates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [107]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)

What is `atemp_level`?

In [109]:
print(atemp_level)


0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]


Here is a random sample of `atemp_level`:

In [112]:
atemp_level.sample(5)

9655     (-0.001, 0.25]
3117        (0.5, 0.75]
4233        (0.5, 0.75]
9180        (0.25, 0.5]
15012       (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [115]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)

14611    warm
15692    mild
16734    mild
13996    warm
16003    mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [140]:
bikes['atemp_level'] = atemp_level
len(bikes[(bikes['atemp_level'] == 'mild') & (bikes['season'] == 2)])


1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

>




---



---



> > > > > > > > > © 2025 Institute of Data


---



---



